In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/GTAI

/content/drive/.shortcut-targets-by-id/1Tcv2eVz-MZJi_SaLgkqfQpTgVhvwlFuJ/GTAI


In [3]:
ls

aggiorna.sh                   FinalProject.ipynb  preprocess.py
attention_nn_steering_angle/  libraries.py        __pycache__/
baseFunctions.py              LICENSE             README.md
convertModelToONNX.py         models_folder/      test.py
Data/                         models.py           train_CNN.py
dataset/                      Papers/             train_Inception.py


In [4]:
!pip install timm
import torch
from torchvision import models
import torch.nn as nn
import torch.nn.functional as F
from libraries import *
import baseFunctions as bf
from models import CNN, Trainer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 549 kB 7.8 MB/s 
     |████████████████████████████████| 182 kB 60.6 MB/s 


In [5]:
class TLearning(nn.Module):
    def __init__(self,device):
        super(TLearning,self).__init__()
        self.device=device
        self.ResNet = models.resnet50(pretrained=True, progress=True)
        self.fc1 = nn.Linear(in_features =1000, out_features = 512, bias=True)
        self.fc2 = nn.Linear(in_features = 512, out_features = 256, bias=True)
        self.fc3 = nn.Linear(in_features = 256, out_features = 64, bias=True)
        self.fc4 = nn.Linear(in_features = 64, out_features = 1, bias=True)
    def forward(self, Input):
        image = self.ResNet(Input)
        # input size = (1,3,224,224)-(Batches, Channels, Height, Width)
        # output size = (1, 1000)-(Batches, Feature)
        image = F.relu(self.fc1(image))
        # input size = (1,1000)-(batches, features)
        # output size = (1,512)-(batches, features)
        image = F.relu(self.fc2(image))
        # input size = (1,512)-(batches, features)
        # output size = (1,256)-(batches, features)
        image = F.relu(self.fc3(image))
        # input size = (1,256)-(batches, features)
        # output size = (1,64)-(batches, features)
        angle = self.fc4(image)
        # input size = (1,64)-(batches, features
        # output size = (1,1)-(batches, features)
        return angle

In [ ]:
if __name__ == '__main__':
    
    if not torch.cuda.is_available():
        device=torch.device("cpu")
        print("Current device:", device)
    else:
        device=torch.device("cuda")
        print("Current device:", device, "- Type:", torch.cuda.get_device_name(0))
        bf.get_memory()
        
    
    #Path per i salvataggi dei checkpoints
    #SINTASSI: ./Data/models/NOME_MODELLO/etc...
    CKP_DIR = "./Data/models/TLearning/checkpoint/"
    SCORE_DIR = "./Data/models/TLearning/scores/"
    SCORE_FILE = 'history_score.pkl'
    
        
    train_dataset = bf.GTADataset("data_train.csv", DATA_ROOT_DIR, bf.preprocess)
    test_dataset = bf.GTADataset("data_test.csv", DATA_ROOT_DIR, bf.preprocess, train=False)
    
    train_dl = DataLoader(train_dataset, 
                            batch_size=256, 
                            sampler=bf.SteeringSampler("./Data/data_train.csv"), 
                            num_workers=10)

    
    test_dl = DataLoader(test_dataset, 
                            batch_size=1, 
                            num_workers=10)


    cnn = TLearning(device = device).to(device) #qui inserire modello da trainare
    
    
    trainer = Trainer(cnn, train_dl, 
                      ckp_dir = CKP_DIR, 
                      score_dir = SCORE_DIR, 
                      score_file = SCORE_FILE)

    
    trainer.train_model(max_epoch=35, 
                        steps_per_epoch=1,
                        lr=0.01,
                        gamma = 0.8,
                        weight_decay=1e-6,
                        log_step=1, 
                        ckp_save_step = 5,
                        ckp_epoch=0)
    
    
    print('Starting test...')
    trainer.test_model(test_dl)
    

#== Best Result ==
#Total Test Loss:  0.0909 --- MAE:  7.8666




#Check loss history
#scores = bf.read_object("./Data/models/CNN/scores/00055_history_score.pkl")

plt.plot(scores['loss_tot_train'][15:])

plt.plot(scores['MAE_train'][15:])

Current device: cpu


/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also 

  0%|          | 0.00/97.8M [00:00<?, ?B/s]

Start Training...

---> Epoch 001/035 <--- 


  0%|          | 0/1 [00:00<?, ?it/s]